# Collecting arxiv abstract information

In this notebook I collect arxiv information from the abstract pages. No links are followed, thus no crawling is needed. 

In [1]:
from  scrapy import Selector
import requests
import time as time
import numpy as np
import pandas as pd


In [2]:
#function to go trough single one 
def get_single_abstract(url,silent=False):
    check=requests.get(url)
    #status exist if 200 or below 400
    if check.status_code==200:
        #result list which is collects
        results=[]
        id_is=url.split('/')[-1]
        #website id
        results.append(id_is)
        #abstract number excluding version
        if id_is.find('v')>-1:
            t=id_is.split('v')
            results.append(t[0])
            #whetehr final 
            results.append("False")
        else:
            results.append(id_is)
            results.append("True")            
        if silent==False:
            print(f"{url} can be accessed")
        html=check.content
        sel=Selector(text=html)
        #abstract selection
        sel2=sel.xpath('//blockquote//text()')
        #extract content
        p2=sel2.extract()
        #get abstract content replace new line with space 
        abstract=p2[2].replace('\n',' ').strip()
        if silent==False:
            print(abstract)
        #get title
        sel3=sel.xpath('//*[@class="title mathjax"]//text()')
        p3=sel3.extract()
        #get title content replace new line with space 
        title=p3[1].replace('\n',' ').strip()
        results.append(title)
        results.append(abstract)        
        if silent==False:
            print(title)
        #select authors
        sel4=sel.xpath('//*[@class="authors"]//text()')
        p4=sel4.extract()
        #get elements which are not authors or comma or empty elements
        authors=[]
        for x in p4:
            if x!="Authors:" and x!=', ':
                y=x.strip()
                if y!='':
                    authors.append(x.strip())
        results.append(len(authors))   
        results.append(authors)          
        if silent==False:
            print(len(authors),authors)
        #get comments
        sel5=sel.xpath('//*[@class="tablecell comments mathjax"]//text()')
        #extract them 
        if len(sel5)==1:
            comments=sel5.extract()[0].strip()
        else:
            comments=None
        if silent==False:
            #print them 
            print(comments)
        results.append(comments)              
        #subject primary
        sel6=sel.xpath('//*[@class="primary-subject"]//text()')
        prim_subject=sel6.extract()[0]
        results.append(prim_subject)          
        if silent==False:
            #print them 
            print(prim_subject)
        #all subjects (doubles previous  to be figured with no other how to combine them )
        sel7=sel.xpath('//*[@class="tablecell subjects"]//text()')
        p7=sel7.extract()
        #exclude first is new line second is primary gotten above 
        if len(p7)==3:
            subjects=p7[2].replace(";","").replace("'","")
            #split them 
            s_subjects_prel=subjects.split(")")
            #adds back ) and excludes last one 
            n_subjects=len(s_subjects_prel)
            s_subjects=[]
            for i in range(len(s_subjects_prel)-1):
                s_subjects.append(s_subjects_prel[i]+")")
            if silent==False:    
                print(s_subjects)
        else:
            s_subjects=None
            n_subjects=0
        results.append(n_subjects)
        results.append(s_subjects)           
        #should be split by ; when not empty
        if silent==False:
            print(n_subjects,s_subjects)   
        #to add submission time? and size?    
        #getting version string if not export version 
        if url.find('export')>-1:
            sel8=sel.xpath('//*[@class="submission-history"]//b/text()')
        else:
            sel8=sel.xpath('//*[@class="submission-history"]//strong/text()')
        sel8b=sel.xpath('//*[@class="submission-history"]/text()')
        #version number
        vlast=sel8.extract()[0].strip('[').strip(']').strip('v')
        if silent==False:
            print(vlast)
        #version properties
        vprop=sel8b.extract()
        vsel=vprop[-2].strip()
        if silent==False:
            print(vsel)
        #append vesrion number number
        results.append(int(vlast))  
        #append version properties
        results.append(vsel)
        return results, 0
    else:
        print(f"{url} cannot be accessed, status is {check.status_code}")
        return None, 1

In [3]:
#some special (big author list) retracted first
urls=['https://export.arxiv.org/abs/1703.00052','https://export.arxiv.org/abs/2010.10965','https://export.arxiv.org/abs/2010.10966','https://export.arxiv.org/abs/2010.10967']
#make them automatic , 404 when obviously not extsing
#403 access denied when to many thus now xport used 

In [4]:
#the long author list cause problems not clear whether needed to solve 
start_time=time.time()
for url in urls:
    # list of strings
    res,check=get_single_abstract(url)
stop_time=time.time()
print(f"{len(urls)} sites needed {np.round(stop_time-start_time,3)} seconds")
#not vsisible authors are visisble here but also not numbers 

https://export.arxiv.org/abs/1703.00052 can be accessed
We describe the Sloan Digital Sky Survey IV (SDSS-IV), a project encompassing three major spectroscopic programs. The Apache Point Observatory Galactic Evolution Experiment 2 (APOGEE-2) is observing hundreds of thousands of Milky Way stars at high resolution and high signal-to-noise ratio in the near-infrared. The Mapping Nearby Galaxies at Apache Point Observatory (MaNGA) survey is obtaining spatially-resolved spectroscopy for thousands of nearby galaxies (median redshift of z = 0.03). The extended Baryon Oscillation Spectroscopic Survey (eBOSS) is mapping the galaxy, quasar, and neutral gas distributions between redshifts z = 0.6 and 3.5 to constrain cosmology using baryon acoustic oscillations, redshift space distortions, and the shape of the power spectrum. Within eBOSS, we are conducting two major subprograms: the SPectroscopic IDentification of eROSITA Sources (SPIDERS), investigating X-ray AGN and galaxies in X-ray clusters

https://export.arxiv.org/abs/2010.10965 can be accessed
Theoretical study is presented for a spinor Bose-Einstein condensate, whose two components are coupled by copropagating Raman beams with different orbital angular momenta. The investigation is focused on the behavior of the ground state of this condensate, depending on the atom-light coupling strength. By analyzing the ground state, we have identified a number of quantum phases, which reflect the symmetries of the effective Hamiltonian and are characterized by the specific structure of the wave function. In addition to the well-known stripe, polarized and zero-momentum phases, our results show that the system can support phases, whose wave function contains a complex vortex molecule. Such molecule plays an important role in the continuous phase transitions of the system. The predicted behavior of vortex-molecule phases can be examined in cold-atom experiments using currently existing techniques.
Symmetry breaking and phase transit

In [5]:
print(res)

['2010.10967', '2010.10967', 'True', 'Safe Handover in Mixed-Initiative Control for Cyber-Physical Systems', 'For mixed-initiative control between cyber-physical systems (CPS) and its users, it is still an open question how machines can safely hand over control to humans. In this work, we propose a concept to provide technological support that uses formal methods from AI -- description logic (DL) and automated planning -- to predict more reliably when a hand-over is necessary, and to increase the advance notice for handovers by planning ahead of runtime. We combine this with methods from human-computer interaction (HCI) and natural language generation (NLG) to develop solutions for safe and smooth handovers and provide an example autonomous driving scenario. A study design is proposed with the assessment of qualitative feedback, cognitive load and trust in automation.', 19, ['Frederik Wiehr', ',', 'Anke Hirsch', ',', 'Florian Daiber', ',', 'Antonio Kruger', ',', 'Alisa Kovtunova', ',',

Looks now good enough at least for standard author lists. Now I build a function which gets all these for all urls starting with one and getting also the previous versions and the following numbers, until some break condition is reached. 

In [6]:
# input starturl, any no v abstratc works, it does not carry over the next month, max_number of primary, number of hours run
#whether get_single_abstract is silent

def get_many(starturl,max_number=20,max_h=1,silent=True):
    start_time=time.time()
    counter=0
    # list of strings
    print(starturl)
    res,check=get_single_abstract(starturl,silent=silent)
    if check==0:
        df2=pd.DataFrame(res,index=['Id','Abstract-number','Final','Title','Abstract','Author_number','Authors','Comments','Prim_subject','n_sec_subjects','sec_subjects','version_number','version_properties']).T
        counter+=1
    check_time=time.time()
    #as long as time o.k. and iteration o.k. and scrap worked
    while (check_time-start_time)/3600<max_h and counter<max_number and check==0:
        print(counter)
        new_url=starturl[:-5]+str(int(starturl[-5:])+counter).zfill(5)
        print(new_url)
        res,check=get_single_abstract(new_url,silent=silent)
        check_time=time.time()
        if check==0:
            df=pd.DataFrame(res,index=['Id','Abstract-number','Final','Title','Abstract','Author_number','Authors','Comments','Prim_subject','n_sec_subjects','sec_subjects','version_number','version_properties']).T
            df2=pd.concat([df2,df],axis=0,ignore_index=True)
            counter+=1          
            vcounter=0
            #looping over versions
            print(f" version number is {int(df.loc[0,['version_number']])}")
            if int(df.loc[0,['version_number']])>1:
                while int(df.loc[0,['version_number']])>1:            
                    urlb=new_url+'v'+str(int(df.loc[0,['version_number']]-1))
                    print(urlb)
                    res,check=get_single_abstract(urlb,silent=silent)
                    df=pd.DataFrame(res,index=['Id','Abstract-number','Final','Title','Abstract','Author_number','Authors','Comments','Prim_subject','n_sec_subjects','sec_subjects','version_number','version_properties']).T        
                    print(f" version number is {int(df.loc[0,['version_number']])}")
                    df2=pd.concat([df2,df],axis=0,ignore_index=True)       
    return df2

In [7]:
df3=get_many('https://export.arxiv.org/abs/2005.00001',max_number=100,max_h=10,silent=True)
df3.to_csv('abstract_2005_part1.csv', sep=',')


https://export.arxiv.org/abs/2005.00001
1
https://export.arxiv.org/abs/2005.00002
 version number is 1
2
https://export.arxiv.org/abs/2005.00003
 version number is 2
https://export.arxiv.org/abs/2005.00003v1
 version number is 1
3
https://export.arxiv.org/abs/2005.00004
 version number is 1
4
https://export.arxiv.org/abs/2005.00005
 version number is 2
https://export.arxiv.org/abs/2005.00005v1
 version number is 1
5
https://export.arxiv.org/abs/2005.00006
 version number is 3
https://export.arxiv.org/abs/2005.00006v2
 version number is 2
https://export.arxiv.org/abs/2005.00006v1
 version number is 1
6
https://export.arxiv.org/abs/2005.00007
 version number is 1
7
https://export.arxiv.org/abs/2005.00008
 version number is 3
https://export.arxiv.org/abs/2005.00008v2
 version number is 2
https://export.arxiv.org/abs/2005.00008v1
 version number is 1
8
https://export.arxiv.org/abs/2005.00009
 version number is 2
https://export.arxiv.org/abs/2005.00009v1
 version number is 1
9
https://expor

 version number is 1
72
https://export.arxiv.org/abs/2005.00073
 version number is 2
https://export.arxiv.org/abs/2005.00073v1
 version number is 1
73
https://export.arxiv.org/abs/2005.00074
 version number is 2
https://export.arxiv.org/abs/2005.00074v1
 version number is 1
74
https://export.arxiv.org/abs/2005.00075
 version number is 3
https://export.arxiv.org/abs/2005.00075v2
 version number is 2
https://export.arxiv.org/abs/2005.00075v1
 version number is 1
75
https://export.arxiv.org/abs/2005.00076
 version number is 2
https://export.arxiv.org/abs/2005.00076v1
 version number is 1
76
https://export.arxiv.org/abs/2005.00077
 version number is 1
77
https://export.arxiv.org/abs/2005.00078
 version number is 2
https://export.arxiv.org/abs/2005.00078v1
 version number is 1
78
https://export.arxiv.org/abs/2005.00079
 version number is 1
79
https://export.arxiv.org/abs/2005.00080
 version number is 1
80
https://export.arxiv.org/abs/2005.00081
 version number is 8
https://export.arxiv.org/

Looking into some content. 

In [13]:
#Combiinng several files. 
df1=pd.read_csv("abstract_2001_part1.csv")
df2=pd.read_csv("abstract_2001_part2.csv")
df3=pd.read_csv("abstract_2001_part3.csv")
df4=pd.read_csv("abstract_2002_part1.csv")
df5=pd.read_csv("abstract_2003_part1.csv")
df6=pd.read_csv("abstract_2004_part1.csv")
df7=pd.read_csv("abstract_2005_part1.csv")
dfall=pd.concat([df1,df2,df3,df4,df5,df6,df7],axis=0,ignore_index=True)
print(dfall.shape,dfall.loc[:,['Id','Abstract-number','Final','version_number']],dfall.version_number.describe())

(92339, 14)                  Id  Abstract-number  Final  version_number
0        2001.00001       2001.00001   True               2
1        2001.00002       2001.00002   True               1
2        2001.00003       2001.00003   True               3
3      2001.00003v2       2001.00003  False               2
4      2001.00003v1       2001.00003  False               1
...             ...              ...    ...             ...
92334    2005.00098       2005.00098   True               1
92335    2005.00099       2005.00099   True               2
92336  2005.00099v1       2005.00099  False               1
92337    2005.00100       2005.00100   True               2
92338  2005.00100v1       2005.00100  False               1

[92339 rows x 4 columns] count    92339.000000
mean         1.669728
std          1.289139
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max         44.000000
Name: version_number, dtype: float64


Select number of final version abstracts. 

In [14]:
dfs=dfall[(dfall.Final==True)]
print(dfs.shape)

(54482, 14)


More than 54000 different abstract papers. Looking on the main categories. 

In [20]:
print(dfs.Prim_subject.value_counts()[0:59])

Computer Vision and Pattern Recognition (cs.CV)               3163
Machine Learning (cs.LG)                                      3004
Quantum Physics (quant-ph)                                    1903
Computation and Language (cs.CL)                              1443
High Energy Physics - Phenomenology (hep-ph)                  1388
Analysis of PDEs (math.AP)                                    1295
Materials Science (cond-mat.mtrl-sci)                         1193
Mesoscale and Nanoscale Physics (cond-mat.mes-hall)           1172
Astrophysics of Galaxies (astro-ph.GA)                        1161
High Energy Physics - Theory (hep-th)                         1081
Optimization and Control (math.OC)                            1035
General Relativity and Quantum Cosmology (gr-qc)              1031
Signal Processing (eess.SP)                                   1008
Combinatorics (math.CO)                                       1001
High Energy Astrophysical Phenomena (astro-ph.HE)             